# Save and retrieve jobs

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to file and retrieve results of past jobs from IBM Quantum&trade; so your next session can continue where you left off.

## Retrieve jobs from IBM Quantum

IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs through the [Jobs dashboard](https://quantum.ibm.com/jobs).

To find a job programatically, use the [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) method. By default, this returns the most recent jobs, but you can also filter jobs by backend name, creation date, and more. The following cell finds any jobs submitted in the last three months. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [1]:
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

service = QiskitRuntimeService()
jobs_in_last_three_months = service.jobs(created_after=three_months_ago)
jobs_in_last_three_months[:3]  # show first three jobs

[<RuntimeJob('crmfqhetdjr0008dzb7g', 'estimator')>,
 <RuntimeJob('crmfq4m14ys00088aqk0', 'estimator')>,
 <RuntimeJob('crm3r8stdjr0008dyqp0', 'qasm3-runner')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) in the API documentation.

Once you have the job ID, use the [`QiskitRuntimeService.job`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#job) method to retrieve it.

In [2]:
# Get ID of most recent successful job for demonstration.
# This will not work if you've never successfully run a job.
job_id = service.jobs(pending=False)[0].job_id()
print(job_id)

crmfqhetdjr0008dzb7g


In [3]:
retrieved_job = service.job(job_id)
retrieved_job.result()

PrimitiveResult([PubResult(data=DataBin<99>(evs=[ 0.64570064  0.31953125  0.15481172  0.15026362  0.1015873   0.06350626
  0.02247191  0.03113208  0.02272727 -0.01854839  0.02457757 -0.02230769
  0.02293578 -0.01092044 -0.02472952 -0.00944882  0.03767123  0.02166934
  0.01305057 -0.01098901  0.01068702  0.01762821  0.01932773  0.01294498
 -0.01468315 -0.01232877 -0.03030303  0.01155462  0.12745098 -0.02332815
  0.00245098  0.05736434 -0.00078616 -0.01820941  0.02724359  0.03257576
 -0.0229709   0.01394052  0.03928571  0.02599388 -0.0015625   0.02323718
 -0.01215805 -0.00234742  0.00466563  0.03106061  0.01630435  0.0464527
  0.         -0.00151745  0.02841782  0.01697531  0.01351351 -0.01018676
  0.00152672  0.00622084  0.0060698   0.02579666  0.02        0.01148545
  0.02586207  0.00381098 -0.01461538  0.04876161  0.00076805 -0.00608696
 -0.00813743  0.         -0.03056027 -0.02665245  0.01660156 -0.16216216
 -0.064      -0.01295732 -0.01930894 -0.01829268  0.02163833  0.00496689
  0.

## Save results to disk

You may also want to save results to disk. To do this, use Python's built-in JSON library with Qiskit IBM Runtime's encoders.

In [4]:
import json
from qiskit_ibm_runtime import RuntimeEncoder
with open("result.json", "w") as file:
    json.dump(retrieved_job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [5]:
from qiskit_ibm_runtime import RuntimeDecoder
with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result

PrimitiveResult([PubResult(data=DataBin<99>(evs=[ 0.64570064  0.31953125  0.15481172  0.15026362  0.1015873   0.06350626
  0.02247191  0.03113208  0.02272727 -0.01854839  0.02457757 -0.02230769
  0.02293578 -0.01092044 -0.02472952 -0.00944882  0.03767123  0.02166934
  0.01305057 -0.01098901  0.01068702  0.01762821  0.01932773  0.01294498
 -0.01468315 -0.01232877 -0.03030303  0.01155462  0.12745098 -0.02332815
  0.00245098  0.05736434 -0.00078616 -0.01820941  0.02724359  0.03257576
 -0.0229709   0.01394052  0.03928571  0.02599388 -0.0015625   0.02323718
 -0.01215805 -0.00234742  0.00466563  0.03106061  0.01630435  0.0464527
  0.         -0.00151745  0.02841782  0.01697531  0.01351351 -0.01018676
  0.00152672  0.00622084  0.0060698   0.02579666  0.02        0.01148545
  0.02586207  0.00381098 -0.01461538  0.04876161  0.00076805 -0.00608696
 -0.00813743  0.         -0.03056027 -0.02665245  0.01660156 -0.16216216
 -0.064      -0.01295732 -0.01930894 -0.01829268  0.02163833  0.00496689
  0.

In [6]:
# Cleanup the file we created (this cell should be hidden from the user)
import pathlib
pathlib.Path('result.json').unlink()